In [2]:
import pandas as pd
import numpy as np

In [46]:
import yfinance as yf

In [18]:
df=pd.read_csv('/Users/hrishityelchuri/TBTFW_task/TBTFW_task/data/AAPL.csv')

In [53]:
start_date = '2021-10-09' #500 stock days before 2023-02-21
end_date = '2024-02-16'

In [54]:
aapl=yf.download('AAPL', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [55]:
aapl

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-11,142.270004,144.809998,141.809998,142.809998,140.811279,64452200
2021-10-12,143.229996,143.250000,141.039993,141.509995,139.529465,73035900
2021-10-13,141.240005,141.399994,139.199997,140.910004,138.937881,78762700
2021-10-14,142.110001,143.880005,141.509995,143.759995,141.747955,69907100
2021-10-15,143.770004,144.899994,143.509995,144.839996,142.812881,67940300
...,...,...,...,...,...,...
2024-02-09,188.649994,189.990005,188.000000,188.850006,188.850006,45155200
2024-02-12,188.419998,188.669998,186.789993,187.149994,187.149994,41781900
2024-02-13,185.770004,186.210007,183.509995,185.039993,185.039993,56529500


In [19]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-02-21,150.199997,151.300003,148.410004,148.479996,147.691666,58867200
1,2023-02-22,148.869995,149.949997,147.160004,148.910004,148.119370,51011300
2,2023-02-23,150.089996,150.339996,147.240005,149.399994,148.606766,48394200
3,2023-02-24,147.110001,147.190002,145.720001,146.710007,145.931061,55469600
4,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.134628,44998500
...,...,...,...,...,...,...,...
245,2024-02-12,188.419998,188.669998,186.789993,187.149994,187.149994,41781900
246,2024-02-13,185.770004,186.210007,183.509995,185.039993,185.039993,56529500
247,2024-02-14,185.320007,185.529999,182.440002,184.149994,184.149994,54630500
248,2024-02-15,183.550003,184.490005,181.350006,183.860001,183.860001,65434500


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       250 non-null    object 
 1   Open       250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Close      250 non-null    float64
 5   Adj Close  250 non-null    float64
 6   Volume     250 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [21]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,250.000000,250.000000,250.000000,250.000000,250.000000,2.500000e+02
mean,178.287761,179.777960,176.983240,178.492360,177.941889,5.695244e+07
std,12.843690,12.698113,12.786402,12.725368,12.866094,1.591466e+07
min,144.380005,146.710007,143.899994,145.309998,144.538513,2.404830e+07
25%,171.342502,173.157501,170.552502,171.869999,171.221340,4.693145e+07
50%,179.590003,180.794998,177.590004,179.410004,178.884682,5.322830e+07
75%,189.312500,189.990005,187.737500,189.352497,188.939693,6.221395e+07
max,198.020004,199.619995,197.000000,198.110001,197.857529,1.282567e+08


In [22]:
df.duplicated().sum()

0

In [56]:
# Calculate the moving averages
aapl['50-day MA'] = aapl['Close'].rolling(window=50).mean()
aapl['500-day MA'] = aapl['Close'].rolling(window=500).mean()
aapl['20-day MA'] = aapl['Close'].rolling(window=20).mean()
aapl['200-day MA'] = aapl['Close'].rolling(window=200).mean()
aapl['10-day MA'] = aapl['Close'].rolling(window=10).mean()
aapl['5-day MA'] = aapl['Close'].rolling(window=5).mean()

In [61]:
aapl

,Open,High,Low,Close,Adj Close,Volume,50-day MA,500-day MA,20-day MA,200-day MA,10-day MA,5-day MA
Date,,,,,,,,,,,,
2021-10-11,142.270004,144.809998,141.809998,142.809998,140.811279,64452200,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-12,143.229996,143.250000,141.039993,141.509995,139.529465,73035900,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-13,141.240005,141.399994,139.199997,140.910004,138.937881,78762700,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-14,142.110001,143.880005,141.509995,143.759995,141.747955,69907100,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15,143.770004,144.899994,143.509995,144.839996,142.812881,67940300,NaN,NaN,NaN,NaN,NaN,142.765997
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-09,188.649994,189.990005,188.000000,188.850006,188.850006,45155200,190.4794,164.60566,189.150999,183.06770,188.044,188.712003
2024-02-12,188.419998,188.669998,186.789993,187.149994,187.149994,41781900,190.4350,164.64220,189.212499,183.18465,187.586,188.606003
2024-02-13,185.770004,186.210007,183.509995,185.039993,185.039993,56529500,190.3368,164.66670,189.282999,183.26780,187.286,187.754001


In [62]:
aapl.to_excel('aapl.xlsx')

In [34]:
# Generate buy and sell signals
df['Buy Signal'] = ((df['50-day MA'] > df['500-day MA']) & (df['20-day MA'] > df['200-day MA'])).astype(int)
df['Sell Signal'] = ((df['20-day MA'] < df['200-day MA']) & (df['10-day MA'] < df['5-day MA'])).astype(int)

In [35]:
# Track buy and sell positions
buy_position = False
sell_position = False
positions = []

In [36]:
for index, row in df.iterrows():
    if row['Buy Signal'] == 1:
        if not buy_position:
            buy_position = True
            positions.append(('Buy', row['Date'], row['Close']))
    elif row['Sell Signal'] == 1:
        if not sell_position:
            sell_position = True
            positions.append(('Sell', row['Date'], row['Close']))
    else:
        if buy_position:
            buy_position = False
            positions.append(('Close Buy', row['Date'], row['Close']))
        if sell_position:
            sell_position = False
            positions.append(('Close Sell', row['Date'], row['Close']))

In [37]:
# Print the buy/sell positions
print("Buy/Sell Positions:")
for position in positions:
    print(position)

Buy/Sell Positions:
